In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from gensim.models.keyedvectors import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
path_to_train = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
path_to_test = tf.keras.utils.get_file('test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [3]:
train_text = open(path_to_train, 'rb').read().decode(encoding='utf-8')
test_text = open(path_to_test, 'rb').read().decode(encoding='utf-8')

In [4]:
tmp = KeyedVectors.load('./data/emb_en2ko_noun.w2v')
tmp_2 = KeyedVectors.load('./data/emb_en2ko_verb.w2v')

In [41]:
tmp_2.wv.most_similar('seoul', topn=50)

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


[('seoulh', 0.899796187877655),
 ('seoulite', 0.8980000019073486),
 ('seoun', 0.8858004808425903),
 ('seoula', 0.8574132323265076),
 ('seoultech', 0.8443858623504639),
 ('seoulbus', 0.8298246264457703),
 ('seoulmetro', 0.8171312808990479),
 ('cineseoul', 0.8098125457763672),
 ('seokbul', 0.8047872185707092),
 ('seoulfortress', 0.7911849021911621),
 ('seoil', 0.789262056350708),
 ('seouruiryowon', 0.789069414138794),
 ('seokjeon', 0.7855755686759949),
 ('seonamsa', 0.7844101190567017),
 ('seogu', 0.7815464735031128),
 ('seonam', 0.7800287008285522),
 ('sportsseoul', 0.778380274772644),
 ('seongpo', 0.777580976486206),
 ('seon', 0.7766729593276978),
 ('seosan', 0.7757962942123413),
 ('seokyungjoo', 0.7729594707489014),
 ('seoksu', 0.7723531723022461),
 ('seongnam', 0.7723338007926941),
 ('seoktap', 0.7715747952461243),
 ('seokgye', 0.7690640687942505),
 ('seok', 0.7689398527145386),
 ('seohocheon', 0.7686943411827087),
 ('seonjeongneung', 0.7658331990242004),
 ('seoulutd', 0.762290000915

In [5]:
import re
def clean_str(string):
    string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\'{2,}", "\'", string)
    string = re.sub(r"\'", "", string)
    return string.lower()
train_text_x = [row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t') > 0]
train_text_x = [clean_str(sentence) for sentence in train_text_x]
sentences = [sentence.split(' ') for sentence in train_text_x]
for i in range(5):
    print(sentences[i])

['아', '더빙', '진짜', '짜증나네요', '목소리']
['흠', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나']
['너무재밓었다그래서보는것을추천한다']
['교도소', '이야기구먼', '솔직히', '재미는', '없다', '평점', '조정']
['사이몬페그의', '익살스런', '연기가', '돋보였던', '영화', '!', '스파이더맨에서', '늙어보이기만', '했던', '커스틴', '던스트가', '너무나도', '이뻐보였다']


In [6]:
train_text_x[:100]

['아 더빙 진짜 짜증나네요 목소리',
 '흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 솔직히 재미는 없다 평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화 ! 스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화 별반개도 아까움 ',
 '원작의 긴장감을 제대로 살려내지못했다 ',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지 정말 발로해도 그것보단 낫겟다 납치 감금만반복반복 이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데 \\? 꽤 볼만한데 헐리우드식 화려함에만 너무 길들여져 있나 \\? ',
 '걍인피니트가짱이다 진짜짱이다 ',
 '볼때마다 눈물나서 죽겠다90년대의 향수자극 ! ! 허진호는 감성절제멜로의 달인이다 ',
 '울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해',
 '담백하고 깔끔해서 좋다 신문기사로만 보다 보면 자꾸 잊어버린다 그들도 사람이었다는 것을 ',
 '취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지',
 ' 냥 매번 긴장되고 재밋음 ',
 '참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다 그냥 까고싶어서 안달난것처럼 보인다',
 '굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐',
 '이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드 ! ! ',
 '약탈자를 위한 변명 , 이라 저놈들은 착한놈들 절대 아닌걸요 ',
 '나름 심오한 뜻도 있는 듯 그냥 학생이 선생과 놀아나는 영화는 절대 아님',
 '보면서 웃지 않는 건 불가능하다',
 '재미없다 지루하고 같은 음식 영화인데도 바베트의 만찬하고 넘 차이남 바베트의 만찬은 이야기도 있고 음식 보는재미도 있는데

In [7]:
tokenizer = Tokenizer(num_words=20000, oov_token="<UNK>")
tokenizer.fit_on_texts(sentences)
train_x = tokenizer.texts_to_sequences(sentences)
train_x = pad_sequences(train_x, padding='post', maxlen=25)

In [8]:
train_y = np.array([[int(row.split('\t')[2])] for row in train_text.split('\n')[1:] if row.count('\t') > 0])
test_y = np.array([[int(row.split('\t')[2])] for row in test_text.split('\n')[1:] if row.count('\t') > 0])

In [9]:
train_x.shape

(150000, 25)

In [10]:
VOCAB_SIZE = len(tokenizer.index_word) + 1
EMBEDDING_DIM = 300
INPUT_LENGTH = 25

embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

# tokenizer에 있는 단어 사전을 순회하면서 word2vec의 300차원 vector를 가져옵니다
for word, idx in tokenizer.word_index.items():
    embedding_vector = tmp[word] if word in tmp else None
    if embedding_vector is not None:
        embedding_matrix[idx] = embedding_vector

In [11]:
# 토크나이저 만들어야함 
input_x = []

for sentence in sentences:
    tmp_list = []
    for word in sentence:
        try:
            tmp_list.append(tmp.get_vector(word))
        except:
            tmp_list.append((np.zeros((300, ))))
            
    while len(tmp_list) < 25:
        tmp_list.append((np.zeros((300, ))))
    
    input_x.append(tmp_list[:25])
    
input_x = np.array(input_x)

In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(25, 300)))
model.add(tf.keras.layers.LSTM(256, recurrent_dropout=0.1))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               570368    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 586,946
Trainable params: 586,946
Non-trainable params: 0
_________________________________________________________________


In [14]:
train_x.shape

(150000, 25)

In [25]:
history = model.fit(input_x, train_y, epochs=50, batch_size=512, validation_split=0.2)

Train on 120000 samples, validate on 30000 samples
Epoch 1/50
120000/120000 [==============================] - 10s 87us/sample - loss: 0.1486 - accuracy: 0.9186 - val_loss: 0.9299 - val_accuracy: 0.7622
Epoch 2/50
120000/120000 [==============================] - 10s 86us/sample - loss: 0.1474 - accuracy: 0.9194 - val_loss: 0.9663 - val_accuracy: 0.7656
Epoch 3/50
120000/120000 [==============================] - 10s 85us/sample - loss: 0.1448 - accuracy: 0.9206 - val_loss: 0.9553 - val_accuracy: 0.7669
Epoch 4/50
120000/120000 [==============================] - 10s 85us/sample - loss: 0.1448 - accuracy: 0.9204 - val_loss: 1.0370 - val_accuracy: 0.7659
Epoch 5/50
120000/120000 [==============================] - 10s 85us/sample - loss: 0.1427 - accuracy: 0.9212 - val_loss: 0.9198 - val_accuracy: 0.7668
Epoch 6/50
120000/120000 [==============================] - 10s 86us/sample - loss: 0.1435 - accuracy: 0.9208 - val_loss: 0.9621 - val_accuracy: 0.7616
Epoch 7/50
120000/120000 [===========

KeyboardInterrupt: 

In [26]:
input_sentence = "The Way Back wants to be Manchester by the Sea on the hardwood, but it's more like Hoosiers with a hangover."
input_sentence = input_sentence.split(' ')
sample_x = []

for sentence in [input_sentence]:
    tmp_list = []
    for word in sentence:
        try:
            print(word)
            tmp_list.append(tmp.get_vector(word))
        except:
            tmp_list.append((np.zeros((300, ))))
            
    while len(tmp_list) < 25:
        tmp_list.append((np.zeros((300, ))))
    
    sample_x.append(tmp_list[:25])
    
sample_x = np.array(sample_x)

The
Way
Back
wants
to
be
Manchester
by
the
Sea
on
the
hardwood,
but
it's
more
like
Hoosiers
with
a
hangover.


In [27]:
model.predict(sample_x)

array([[0.8839754 , 0.11602467]], dtype=float32)

In [28]:
import pandas as pd

data = pd.read_csv('./data/SST-2/dev.tsv', delimiter='\t')

In [29]:
eval_x = [i.split(' ') for i in list(data['sentence'])]

In [30]:
len(eval_x)

872

In [31]:
x_eval = []

for sentence in eval_x:
    tmp_list = []
    for word in sentence:
        try:
            tmp_list.append(tmp.get_vector(word))
        except:
            tmp_list.append((np.zeros((300, ))))
            
    while len(tmp_list) < 25:
        tmp_list.append((np.zeros((300, ))))
    
    x_eval.append(tmp_list[:25])
    
x_eval = np.array(x_eval)

In [32]:
x_eval.shape

(872, 25, 300)

In [33]:
eval_y = list(data['label'])

In [34]:
model.evaluate(x_eval, np.array(eval_y))

872/872 [==============================] - 0s 327us/sample - loss: 1.7278 - accuracy: 0.5585


[1.7278463512385658, 0.5584862]